# 这是任务二的特征工程代码

## 分割数据集

## AutoML

- 自动特征工程
    - 特征的自动提取
- 特征筛选
- 模型融合
    - AutoGluon
- Stacking


In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [ ]:
def split_off_dataset(off: pd.DataFrame, space=pd.to_timedelta(0), time_ranges=None):
    """
    滑动窗口分割
    :param space: 时间间隔，特征区间之后时间间隔后的一个月作为标签区间，默认为 0
    :param off: 线下数据集
    :param time_ranges: kv，表示特征区间
    :return: 两个列表，特征区间和标签区间，标签区间为特征区间结束间隔space后的一个月
    """
    print("====== off train set info ======")
    print(off.info())
    print()
    label_len = pd.to_timedelta(30, unit='D') # 三十天
    _feats = []
    _labels = []
    for k, v in time_ranges.items():
        time_range = pd.date_range(k, k+v)
        _feats.append(off[off['Date_received'].isin(time_range) |  # 领券日期在 time_range 内
                          ((off['Coupon_id'] == 0) & off['Date'].isin(time_range))]) # 在 time_range 内直接消费
        print(f'features time range {k} : {k+v}, size {len(_feats[len(_feats)-1])}')
        time_range = pd.date_range(k+v+space+pd.to_timedelta(1, 'D'), k+v+space+label_len)
        _labels.append(off[off['Date_received'].isin(time_range)]) # 训练用的标签集合，标签集合不需要采样直接消费的
        print(f'labels time range {k+v+space+pd.to_timedelta(1, "D")} : {k+v+space+label_len},'
              f' size {len(_labels[len(_labels)-1])}')
    del time_range
    print(f'total features size {sum(map(lambda x: len(x), _feats))}')
    print(f'total labels size {sum(map(lambda x: len(x), _labels))}')
    return _feats, _labels